In [27]:
%matplotlib inline

In [28]:
import seaborn as sns
from mvpa2.suite import *
from os.path import join as pjoin
from joblib.parallel import Parallel, delayed
import pandas as pd
from scipy.spatial.distance import pdist, cdist

from itertools import combinations

In [29]:
raiders_data = h5load('/home/swaroop/mvpa_linux_bkup/slhyper/raiders_dartmouth_8ch_mni_gray_large.hdf5.gz')

In [30]:
# run this for hpal_part 1 and 2
hpal_part = 2
hyper_params_fn = '/home/swaroop/mvpa_linux_bkup/slhyper/' + \
'surfslhypmappersmni_raiders_dartmouth_8ch_radius20_lr32_hr128_part{0}_featsel0.7.hdf5.gz'.format(hpal_part)

In [31]:
hyper_params = h5load(hyper_params_fn)

In [32]:
ds_test = raiders_data['ds']

In [33]:
hmappers = hyper_params['hmappers']

In [54]:
len(range(4,1326))

1322

In [55]:
len(range(1326, 2662))

1336

In [52]:
raiders_data['ds']

[<Dataset: 2662x61831@float32, <sa: time_coords,time_indices>, <fa: voxel_indices>, <a: imghdr,imgtype,mapper,voxel_dim,voxel_eldim>>,
 <Dataset: 2662x61831@float32, <sa: time_coords,time_indices>, <fa: voxel_indices>, <a: imghdr,imgtype,mapper,voxel_dim,voxel_eldim>>,
 <Dataset: 2662x61831@float32, <sa: time_coords,time_indices>, <fa: voxel_indices>, <a: imghdr,imgtype,mapper,voxel_dim,voxel_eldim>>,
 <Dataset: 2662x61831@float32, <sa: time_coords,time_indices>, <fa: voxel_indices>, <a: imghdr,imgtype,mapper,voxel_dim,voxel_eldim>>,
 <Dataset: 2662x61831@float32, <sa: time_coords,time_indices>, <fa: voxel_indices>, <a: imghdr,imgtype,mapper,voxel_dim,voxel_eldim>>,
 <Dataset: 2662x61831@float32, <sa: time_coords,time_indices>, <fa: voxel_indices>, <a: imghdr,imgtype,mapper,voxel_dim,voxel_eldim>>,
 <Dataset: 2662x61831@float32, <sa: time_coords,time_indices>, <fa: voxel_indices>, <a: imghdr,imgtype,mapper,voxel_dim,voxel_eldim>>,
 <Dataset: 2662x61831@float32, <sa: time_coords,time_in

In [34]:
# take only the data we need
if hpal_part == 1:
    ds_test = [sd[1326:, ] for sd in ds_test]
else:
    ds_test = [sd[4:1326, ] for sd in ds_test]

In [35]:
# load mask
data_dir = '/home/swaroop/mvpa_linux_bkup/slhyper'
sfs = h5load(pjoin(data_dir, 'cerebrum_mask.hdf5.gz'))

In [36]:
# apply mask
ds_test = [sfs.forward(sd) for sd in ds_test]

In [37]:
# zscore
_ = [zscore(sd, chunks_attr=None) for sd in ds_test]

In [38]:
def hyperalign(mapper, ds):
    tmp = mapper.forward(ds)
    zscore(tmp, chunks_attr=None)
    return tmp

In [39]:
# save voxel_indices
voxel_indices = ds_test[0].fa.voxel_indices

In [40]:
# hyperaligning
ds_test = Parallel(n_jobs=11)(delayed(hyperalign)(hm, sd) for hm, sd in zip(hmappers, ds_test))

In [41]:
# load rois
rois = h5load('../notebooks/manual_rois_20170222_3r_no-overlap_MNI3mm_mask.hdf5')

In [42]:
labels = list(pd.read_csv('roi_coord.csv')['Abbreviation'])

In [43]:
print('Using {0} rois'.format(len(labels)))

Using 30 rois


In [44]:
def compute_rdms(ds, roi, labels):
    rdms = []
    for label in labels:
        print("Computing RDM for {0}".format(label))
        idx = roi[label][1]
        rdms.append(1. - pdist(ds.samples[:, idx], metric='correlation'))
    rdms = Dataset(rdms, sa={'targets': labels})
    assert(rdms.nfeatures == ds.nsamples * (ds.nsamples - 1) / 2)
    return rdms

In [45]:
rdm_all = Parallel(n_jobs=11)(delayed(compute_rdms)(ds, rois, labels) for ds in ds_test)

Computing RDM for EV1 - L
Computing RDM for EV1 - R
Computing RDM for EV2 - L
Computing RDM for EV2 - R
Computing RDM for OccFus - L
Computing RDM for OccFus - R
Computing RDM for EV1 - L
Computing RDM for EV1 - R
Computing RDM for pFus - L
Computing RDM for EV2 - L
Computing RDM for pFus - R
Computing RDM for EV2 - R
Computing RDM for mFus - L
Computing RDM for OccFus - L
Computing RDM for mFus - R
Computing RDM for OccFus - R
Computing RDM for EV1 - L
Computing RDM for EV1 - R
Computing RDM for pFus - L
Computing RDM for pMTG - L
Computing RDM for EV2 - L
Computing RDM for pMTG - R
Computing RDM for pFus - R
Computing RDM for EV2 - R
Computing RDM for aFus - R
Computing RDM for OccFus - L
Computing RDM for mFus - L
Computing RDM for mMTG - L
Computing RDM for OccFus - R
Computing RDM for EV1 - L
Computing RDM for mFus - R
Computing RDM for EV1 - R
Computing RDM for mMTG - R
Computing RDM for pFus - L
Computing RDM for pMTG - L
Computing RDM for EV2 - L
Computing RDM for aMTG - L
Comp

In [46]:
def cross_correlation(a, b):
    return 1. - cdist(a, b, metric='correlation')

In [47]:
roi_rdm = Parallel(n_jobs=32)(delayed(cross_correlation)
                              (rdm_all[isub], rdm_all[jsub]) 
                              for isub, jsub in combinations(range(11), 2))

In [48]:
roi_rdm_avg = np.arctanh(roi_rdm).mean(axis=0)

In [49]:
roi_rdm_avg += roi_rdm_avg.T
roi_rdm_avg /= 2

In [50]:
roi_rdm_avg = np.tanh(roi_rdm_avg)

In [51]:
h5save('roi_rdm_hpal_part{0}.hdf5'.format(hpal_part), roi_rdm_avg)

## Check that the node indices make sense

In [ ]:
# get voxel indices to find correspondence, first load template
cerebrum_mask = fmri_dataset('/home/swaroop/mvpa_linux_bkup/slhyper/MNI/MNI_mask_cerebrum.nii.gz',
                            mask='/home/swaroop/mvpa_linux_bkup/slhyper/MNI/MNI_mask_cerebrum.nii.gz')

In [ ]:
def find_node_indices_dataset(roi_node_indices, template_voxel_indices, dataset_voxel_indices):
    """
    Given roi_node_indices based on the template voxel_indices, return the 
    node indices based on the this dataset voxel indices
    """
    # get voxel indices from template
    these_voxel_indices = template_voxel_indices[roi_node_indices]
    # now find what the node index is in the actual datset
    roi_node_indices_dataset = []
    for vxl in these_voxel_indices:
        roi_node_indices_dataset.extend(np.where(np.all(vxl == dataset_voxel_indices, axis=1))[0])
        
    # assert we get the same number of elements
    # assert(len(roi_node_indices_dataset) == len(roi_node_indices))
    
    return roi_node_indices_dataset

In [ ]:
for key, (center, nodes) in rois.iteritems():
    try:
        nodes_ = find_node_indices_dataset(nodes, cerebrum_mask.fa.voxel_indices, voxel_indices)
    except:
        pass
    print("{0}: {1}/{2}".format(key, len(nodes_), len(nodes)))

In [ ]:
# ok now we have correspondence